<a href="https://colab.research.google.com/github/cabamarcos/Grupo27_practica2/blob/main/Grupo27_practica2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Importación de librerías y datos

Hacemos los imports correspondientes



In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
with open('attrition_available_27.pkl', 'rb') as archivo:
    datos = pickle.load(archivo)

print(datos)

           hrs  absences  JobInvolvement  PerformanceRating  \
1     7.315971      13.0             2.0                4.0   
6     6.450877      17.0             3.0                4.0   
13    8.871421      14.0             2.0                3.0   
28         NaN       6.0             2.0                3.0   
30    9.662808      11.0             2.0                3.0   
...        ...       ...             ...                ...   
4405  8.316921       6.0             3.0                3.0   
4406  5.897197       NaN             2.0                3.0   
4407       NaN      18.0             3.0                4.0   
4408  9.187612       NaN             2.0                3.0   
4409  6.511790      17.0             4.0                3.0   

      EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance   Age  \
1                         3.0              2.0              4.0  31.0   
6                         1.0              3.0              1.0   NaN   
13                      

#2. EDA

Convertimos los datos en un dataframe

In [3]:
df = pd.DataFrame(datos)

Sacamos el número de instancias y el número de atributos

In [4]:
print("Número de instancias:", len(df))
print("Número de atributos:", len(df.columns))

Número de instancias: 4410
Número de atributos: 31


Vemos el tipo de datos de las instacias

In [5]:
print("Tipos de datos:")
print(df.dtypes)

Tipos de datos:
hrs                        float64
absences                   float64
JobInvolvement             float64
PerformanceRating          float64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
Age                        float64
Attrition                   object
BusinessTravel              object
Department                  object
DistanceFromHome           float64
Education                  float64
EducationField              object
EmployeeCount              float64
EmployeeID                 float64
Gender                      object
JobLevel                   float64
JobRole                     object
MaritalStatus               object
MonthlyIncome              float64
NumCompaniesWorked         float64
Over18                      object
PercentSalaryHike          float64
StandardHours                int64
StockOptionLevel           float64
TotalWorkingYears          float64
TrainingTimesLastYear      float64
Year

Podemos ver que la mayoría de datos son float, pero también observamos objetos y números enteros

Vamos a comprobar si hay columnas constantes en estos datos


In [11]:
# Encontrar las columnas constantes
columnas_constantes = datos.columns[datos.nunique() == 1]
print(columnas_constantes)

Index(['EmployeeCount', 'Over18', 'StandardHours'], dtype='object')


Se puede ver que "EmployeeCount", "Over18" y "StandardHours" tienen valores constantes, esto es bastante logico, ya que "EmployeeCount" indica el numero de empleados en la compañía, y este en el momento de sacar los datos, no varía. Podría variar si se vuelven a sacar los datos x días después. 

"Over18", es normal que tenga valores ctes, ya que es un objeto que puede ser o mayor o menor de 18, y sería muy raro que alguien estuviera trabajando en una empresa con menos de 18 años. 

Finalmente "StandardHours" como dice su propio nombre, es el número estandar de horas trabajadas en la compañía. Al ser un número estandar, no es personalizado por lo que siempre es el mismo.

A continuacion se comprobarán los missing values


In [13]:
proporcion_missing = datos.isnull().mean()
print(proporcion_missing)

hrs                        0.205215
absences                   0.202041
JobInvolvement             0.162812
PerformanceRating          0.208390
EnvironmentSatisfaction    0.199320
JobSatisfaction            0.214966
WorkLifeBalance            0.174376
Age                        0.166893
Attrition                  0.000000
BusinessTravel             0.192517
Department                 0.177324
DistanceFromHome           0.169388
Education                  0.162132
EducationField             0.160998
EmployeeCount              0.186395
EmployeeID                 0.180499
Gender                     0.211338
JobLevel                   0.177324
JobRole                    0.177324
MaritalStatus              0.000000
MonthlyIncome              0.210658
NumCompaniesWorked         0.212245
Over18                     0.209524
PercentSalaryHike          0.178685
StandardHours              0.000000
StockOptionLevel           0.160544
TotalWorkingYears          0.203628
TrainingTimesLastYear      0

Se puede observar que en ciertas columnas no hay ni un solo missing value, mientras que en otras, el % de missing values puede llegar hasta el 0,21%, más de un quinto de los datos faltan.